In [1]:
# Import dependencies.
import os
import pandas as pd
import numpy as np

In [2]:
DATA_BASE = '../movie.data'
CREDIT_PATH = os.path.join(DATA_BASE, 'credits.csv')
RATING_PATH = os.path.join(DATA_BASE, 'ratings.csv')
METADATA_PATH = os.path.join(DATA_BASE, 'movies_metadata.csv')
# credits = pd.read_csv(CREDIT_PATH)
# ratings = pd.read_csv(RATING_PATH)
metadata = pd.read_csv(METADATA_PATH)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
metadata.axes

[RangeIndex(start=0, stop=45466, step=1),
 Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
        'imdb_id', 'original_language', 'original_title', 'overview',
        'popularity', 'poster_path', 'production_companies',
        'production_countries', 'release_date', 'revenue', 'runtime',
        'spoken_languages', 'status', 'tagline', 'title', 'video',
        'vote_average', 'vote_count'],
       dtype='object')]

In [3]:
vote_counts = metadata[metadata['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = metadata[metadata['vote_average'].notnull()]['vote_average'].astype('int')

In [4]:
mean_vote = vote_averages.mean()
minimum_vote_requirement = vote_counts.quantile(0.95)

In [12]:
metadata['year'] = pd.to_datetime(metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [15]:
qualified = metadata[(metadata['vote_count'] >= minimum_vote_requirement) & (metadata['vote_count'].notnull()) & (metadata['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [5]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+minimum_vote_requirement) * R) + (minimum_vote_requirement/(minimum_vote_requirement+v) * mean_vote)

In [20]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [21]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",7.897107
2843,Fight Club,1999,9678,8,63.8696,"[{'id': 18, 'name': 'Drama'}]",7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.851924


In [5]:
import tensorflow as tf
from tensorflow import keras

In [22]:
# x consists of v and r
def weighted_rating(x):
    v = x[0]
    R = x[1]
#     return tf.constant((v/(v+minimum_vote_requirement) * R) + (minimum_vote_requirement/(minimum_vote_requirement+v) * mean_vote), dtype=tf.float32)
    return (v/(v+minimum_vote_requirement) * R) + (minimum_vote_requirement/(minimum_vote_requirement+v) * mean_vote)

_input = keras.layers.Input((2,))
output = keras.layers.Lambda(weighted_rating)(_input)
output = keras.layers.Reshape([1,])(output)
model = keras.models.Model(_input, output)
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['accuracy'])

In [20]:
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda_5 (Lambda)            (1,)                      0         
_________________________________________________________________
reshape_4 (Reshape)          (1, 1)                    0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.predict((10, 5))

array([[5.239381]], dtype=float32)

In [16]:
tf.keras.models.save_model(
    model,
    os.path.join('populairity_model', '1'),
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: populairity_model/1/assets
